# Features Engineering for Retail Sales Forecasting

Bu çalışmada [Samir Saci](http://www.linkedin.com/in/samir-saci)'nin çalışması kullanılmıştır. Samir kendi çalışmasında sadece verisetiyle eğitilen model ile yeni feature'lar eklenerek eğitilen modelin çıktılarını karşılaştırmıştır. Ben bu çalışmada zaman serilerinde önemli olan featureları detaylı bir şekilde açıklamaya çalışacağım. Model karşılaştırmasına Samir'in çalışmasında bulabilirsiniz. Samir'in çalışmasına ait linkler aşağıdaki gibidir. Bu linklerden ulaşabilirsiniz.

* [Samir Saci Portfolio](https://samirsaci.com/)
* [Feature Engineering Medium Post](https://s-saci95.medium.com/machine-learning-for-retail-sales-forecasting-features-engineering-4edfee7c9cbc)
* [Feature Engineering Github Repo](https://github.com/samirsaci/ml-forecast-features-eng)

## 1.Kütüphaneleri Yükleme

In [1]:
import joblib
import os
import pickle
from pathlib import Path

###############################################################################
#                Standart Paket ve Pandas Özelleştirmesi                      #
###############################################################################

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.options.mode.chained_assignment = None
import numpy as np
 
###############################################################################
#              Uyarı, Yazdırmaz Özelleştirmesi, Zamanlama                     #
###############################################################################

import warnings
warnings.filterwarnings("ignore")
from IPython.display import Markdown, display # bold printing method
def printmd(string):
    display(Markdown(string))

###############################################################################
#                               Görselleştirme                                #
###############################################################################

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
matplotlib.rcParams.update({'font.size': 30})
plt.tick_params(axis='both', which='major', labelsize=30)
plt.tick_params(axis='both', which='minor', labelsize=30)
%matplotlib inline
from matplotlib.colors import ListedColormap
import seaborn as sns
sns.set(style="darkgrid", palette="muted", color_codes=True)

###############################################################################
#                               Model                                         #
###############################################################################

import lightgbm
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV

## 2.Veri Okuma ve İnceleme

 Verisetine [bu linkten](https://www.kaggle.com/c/m5-forecasting-accuracy) ulaşabilirsiniz.

In [2]:
sales = pd.read_csv('Data/sales_train_evaluation.csv')
calendar = pd.read_csv('Data/calendar.csv')
prices = pd.read_csv('Data/sell_prices.csv')


print("Sales veri seti boyutu: {}".format(sales.shape))
print("Calendar veri seti boyutu: {}".format(calendar.shape))
print("Prices veri seti boyutu: {}".format(prices.shape))

Sales veri seti boyutu: (30490, 1947)
Calendar veri seti boyutu: (1969, 14)
Prices veri seti boyutu: (6841121, 4)


In [3]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1,0,1,4,0,0,5,0,0,0,0,0,0,2,1,2,1,0,0,0,1,1,1,0,0,1,1,1,1

Sales veri setini incelediğimizde ilk karşımıza çıkan görüntü satırında id, item_id, dept_id, cat_id, store_id ve state_id bilgisinin bulunduğu;  kolonunda gün bilgisinin bulunduğu bir pivot table olmasıdır.

Kolon Bilgileri: 
* id: item_id ve store_id birleşimi
* item_id: Satılan Ürün Kodu
* dept_id: Satılan Ürün Departman Kodu
* cat_id: Satılan Ürün Kategori Kodu
* store_id: Satış Yapılan Mağaza Kodu
* state_id: Satış Yapılan Şehir Kodu
* d_: günler


In [4]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


Calender veriseti sales verisetindeki gün bilgisinin detayını vermektedir.

Kolon Bilgileri:
* date: Tarih
* wm_yr_wk: Hafta Kodu
* weekday: Gün İsmi
* wday: Günün Haftadaki Sayısı
* month: Ay
* year: Yıl
* d: Gün Id Kodu
* event_name_1: Güne Ait Özel Olay Bilgisi
* event_type_1: Güne Ait Özel Olay Tipi
* event_name_2: Güne Ait Özel Olay Bilgisi
* event_type_2: Güne Ait Özel Olay Tipi

In [5]:
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58000
1,CA_1,HOBBIES_1_001,11326,9.58000
2,CA_1,HOBBIES_1_001,11327,8.26000
3,CA_1,HOBBIES_1_001,11328,8.26000
4,CA_1,HOBBIES_1_001,11329,8.26000


Prices veriseti hangi tarihte hangi mağazada hangi üründen ne kadar satılmış bilgisini elde edebiliyoruz.

Kolon Bilgileri:
* store_id: Satış Yapılan Mağaza Kodu
* item_id: Satılan Ürün Kodu
* wm_yr_wk: Hafta Kodu
* sell_price: Satış Fiyatı

## 3.Veri Ön İşleme

In [3]:
#Bu fonksiyon memory kullanımını azaltmak adına veri tiplerini güncellemek için oluşturulmuş. Konumuzla alakalı olmadığından bu kısmı detaylandırmıyorum.
# Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        # Integer
        if 'int' in str(t):
            # Check if minimum and maximum are in the limit of int8
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            # Check if minimum and maximum are in the limit of int16
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            # Check if minimum and maximum are in the limit of int32
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            # Choose int64
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        # Float
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        # Object
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

## Apply downcasting
sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

In [4]:
def improve_price(prices):
    # Bir ürünün satıldığı tüm mağazalardaki zamana bağlı satış fiyatı ortalamasını hesaplıyoruz.
    df_mean_store = pd.DataFrame(prices.groupby(['item_id', 'wm_yr_wk'])['sell_price'].mean())
    df_mean_store.columns = ['item_sells_price_avg']
    df_mean_store.reset_index(inplace = True)

    # Bu ortalamaları kullanmak üzere prices verisetine ekliyoruz.
    prices_new = pd.merge(prices, df_mean_store, on=['item_id', 'wm_yr_wk'], how='left', suffixes=('', '_y'))
    prices_new.drop(prices_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

    # Ürünün satış fiyatının diğer mağazalardaki satış fiyat ortalamasıyla olan farkının deltasını hesaplıyoruz. 
    prices_new['delta_price_all_rel'] = (prices_new['sell_price'] - 
                                         prices_new['item_sells_price_avg'])/prices_new['item_sells_price_avg']

    # Ürün fiyatının mağazadaki geçmiş haftanın fiyatı ile karşılaştırılması.
    prices_new['item_store'] = prices_new['item_id'].astype(str) + '_' + prices_new['store_id'].astype(str)
    prices_new['item_store_change'] = prices_new["item_store"].shift() != prices_new["item_store"]
    # Ürün fiyatının mağazadaki geçmiş hafta fiyatıyla farkının deltasını hesaplıyoruz.
    prices_new['delta_price_weekn-1'] = (prices_new['sell_price']-
                                         prices_new['sell_price'].shift(1)).fillna(0)/prices_new['sell_price'].shift(1)
    prices_new['delta_price_weekn-1'] = prices_new['delta_price_weekn-1'].fillna(0) * (prices_new['item_store_change']==0)

    # Mağazadaki departman bazlı fiyat ortalamasını hesaplıyoruz.
    prices_new['dept_id'] = prices_new.item_id.str[:-4]
    df_mean_cat = pd.DataFrame(prices_new.groupby(['dept_id', 'store_id', 'wm_yr_wk'])['sell_price'].mean())
    df_mean_cat.columns = ['dept_sells_price_avg']
    df_mean_cat.reset_index(inplace = True)
    # Departman bazlı ortalamayı kullanmak üzere ana tabloya ekliyoruz.
    prices_new = pd.merge(prices_new, df_mean_cat, on=['dept_id', 'store_id', 'wm_yr_wk']
                          , how='left', suffixes=('', '_y'))
    prices_new.drop(prices_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

    # Cannibalisation(Yamyamlık): Ürün fiyatını aynı kategorideki(deparman) diğer ürünlerin ortalama fiyatıyla karşılaştırma
    prices_new['delta_price_cat_rel'] = (prices_new['sell_price'] - 
                                         prices_new['dept_sells_price_avg'])/prices_new['dept_sells_price_avg']                                               

    # Kullandığımız kolonları siliyoruz.
    prices_new.drop(['item_sells_price_avg', 'item_store_change', 'item_store_change', 'dept_id', 'item_store',
                    'dept_sells_price_avg'], axis = 1, inplace = True)
    
    return prices_new

In [5]:
prices_new = improve_price(prices)
prices_new.head()

,store_id,item_id,wm_yr_wk,sell_price,delta_price_all_rel,delta_price_weekn-1,delta_price_cat_rel
0,CA_1,HOBBIES_1_001,11325,9.57812,0.00000,0.00000,0.57080
1,CA_1,HOBBIES_1_001,11326,9.57812,0.05872,0.00000,0.53174
2,CA_1,HOBBIES_1_001,11327,8.25781,-0.05960,-0.13782,0.32446
3,CA_1,HOBBIES_1_001,11328,8.25781,-0.02400,0.00000,0.32446
4,CA_1,HOBBIES_1_001,11329,8.25781,-0.02400,0.00000,0.32446


In [6]:
# Pivot tablo olan sales tablosunu modelimizde kullanabilmek adına düz tablo formatına çeviriyoruz.
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
             var_name='d', value_name='sold').dropna()

print("df satır sayısı: {:,} ".format(len(df)))
df.head()

df satır sayısı: 59,181,090 


,id,item_id,dept_id,cat_id,store_id,state_id,d,sold
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [7]:
# calender tablosuyla join işlemi 
df = pd.merge(df, calendar, on='d', how='left')
# price tablosuyla join işlemi
df = pd.merge(df, prices_new, on=['store_id','item_id','wm_yr_wk'], how='left') 

# Gün kolonundaki 'd_' ifadesini silip sayısal değere dönüştürüyoruz.
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

# Gereksiz date kolonunu siliyoruz. 
df.drop('date',axis=1,inplace=True)

print("calender ve price datasetleriyle birleştikten sonra df satır sayısı: {:,}".format(len(df)))
df.head()

calender ve price datasetleriyle birleştikten sonra df satır sayısı: 59,181,090


,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,delta_price_all_rel,delta_price_weekn-1,delta_price_cat_rel
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN


In [8]:
# Memory'de yer kaplamaması adına eski dataframeleri siliyoruz.
del sales
del calendar
del prices
del prices_new

## 4.Feature Engineering

Özellik mühendisliği, makine öğrenimi sürecini kolaylaştırmaya yardımcı olan ham verilerden yeni özellikler oluşturarak makine öğrenmesi algoritmalarının tahmin gücünü artırır ve daha iyi sonuçlar elde edilmesini sağlar.
Kısacası, ham verileri seçme, değiştirme ve denetimli öğrenmede kullanılabilecek özelliklere dönüştürme ve veriyi makine öğrenmesi modellerine hazırlama sürecidir. Bildiğiniz gibi, “özellik”, tahmine dayalı bir modelde kullanılabilecek herhangi bir ölçülebilir girdidir. 

[Kaynak](https://medium.com/@ilaydakursun/makine-%C3%B6%C4%9Frenimi-i%C3%A7in-feature-engineering-temel-teknikleri-e0bb4fe8124a)

### 4.1. Label Encoding

Label Encoding, en sık kullanılan feature engineering başlıklarından biridir. Temelde verisetinde bulunan kategorik değerli kolonları modelimizde kullanabilmek adına sayıllaştırmamıza olanak sağlar. Yaptığı işlem kategorik değerli kolonda bulunan her unique değere alfabetik sıraya göre unique bir sayısal değer atamaktır.

In [9]:
# Kategorik veri içeren kolonların veri tiplerini 'category' olarak güncelliyoruz.
df.id = df.id.astype('category')
df.cat_id = df.cat_id.astype('category')
df.item_id = df.item_id.astype('category')
df.dept_id = df.dept_id.astype('category')
df.store_id = df.store_id.astype('category')
df.state_id = df.state_id.astype('category')

In [10]:
# Kategorik verileri sayısal kodları ile dönüştürüyoruz.
cols = df.dtypes.index.tolist()
types = df.dtypes.values.tolist()

for i,type in enumerate(types):
    if type.name == 'category':
        df[cols[i]] = df[cols[i]].cat.codes

df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,delta_price_all_rel,delta_price_weekn-1,delta_price_cat_rel
0,14370,1437,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN
1,14380,1438,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN
2,14390,1439,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN
3,14400,1440,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN
4,14410,1441,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN


### 4.2 Lag Feature

Lag Feature, zaman serisi tahmin problemlerinin denetimli öğrenme problemlerine dönüştürülmesinin klasik yoludur. Temel olarak hedef değişken kullanılır ancak bir süre ile kaydırılır. Diyelim bir şirketin hisse senedi fiyatını tahmin ediyorsunuz. Bir önceki günkü hisse senedi fiyatı tahmin yapmak için önemli değil midir? Başka bir deyişle, t zamanındaki değer , t-1 zamanındaki değerden büyük ölçüde etkilenir. Geçmiş değerler Lag olarak bilinir, dolayısıyla t-1 Lag 1'dir, t-2 Lag 2'dir.

Lag Feature'ı kullanmak konusunda dikkatli olunmalıdır. Çünkü yanlış veya fazla kullanımı overfitting'e sebep olabilir.

[Kaynak](https://www.analyticsvidhya.com/blog/2019/12/6-powerful-feature-engineering-techniques-time-series/)


In [11]:
# Satışları, farklı lag sayıları kadar kaydırarak  yeni kolonlar oluşturuyoruz. Bu kaydırma işlemini her mağazadaki her ürün için ayrı ayrı yapıyoruz.
lags = [1, 8, 21, 34]
for lag in lags:
    df['sold_lag_'+str(lag)] = df.groupby(
        ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
        as_index = False)['sold'].shift(lag).astype(np.float16)
        
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,delta_price_all_rel,delta_price_weekn-1,delta_price_cat_rel,sold_lag_1,sold_lag_8,sold_lag_21,sold_lag_34
0,14370,1437,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14380,1438,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14390,1439,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14400,1440,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14410,1441,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 4.3. Mean Encoding

Özellik Mühendisliği sırasında, kategorik özellikleri sayısala dönüştürme görevine Kodlama denir. OneHotEncoding ve LabelEncoding , FrequencyEncoding gibi kategorik özellikleri ele almanın veya kategorik özellikleri sayılarına göre değiştirmenin çeşitli yolları vardır. Benzer şekilde MeanEncoding'i de kullanabiliriz. 

İşi verimli bir şekilde ancak rastgele bir şekilde yapan Label Encoding aksine, Mean Encoding, soruna daha mantıklı yaklaşmaya çalışır. Özetle, yeni kodlanmış özelliği oluşturmak için temel olarak hedef değişkeni kullanır.

**MeanEncoding'in Artıları:**

- Etiket içindeki bilgileri yakalayın, böylece daha öngörülü özellikler elde edin
- Değişken ve hedef arasında monoton bir ilişki oluşturur

**MeanEncodeg'in Eksileri:**

- Modelde fazla oturmaya neden olabilir.

[Kaynak](https://towardsdatascience.com/why-you-should-try-mean-encoding-17057262cd0)

In [12]:
# Tek kategorik özellik bazında satış ortalamasını alıyoruz.
df['item_sold_avg'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)
df['state_sold_avg'] = df.groupby('state_id')['sold'].transform('mean').astype(np.float16)
df['store_sold_avg'] = df.groupby('store_id')['sold'].transform('mean').astype(np.float16)
df['cat_sold_avg'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
df['dept_sold_avg'] = df.groupby('dept_id')['sold'].transform('mean').astype(np.float16)

# Kombinasyonlu bir şekilde satış ortalamalarını alıyoruz.
df['cat_dept_sold_avg'] = df.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
df['store_item_sold_avg'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['cat_item_sold_avg'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['dept_item_sold_avg'] = df.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['state_store_sold_avg'] = df.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
df['state_store_cat_sold_avg'] = df.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
df['store_cat_dept_sold_avg'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)

### 4.4. Rolling Average(Satış)

Hareketli ortalama, n tane (belirlenen çerçeve) fiyatın ortalamasının alınarak hesaplanan bir göstergedir. Hareketli ortalamalar trend takibi açısından önemli bir gösterge olarak kabul edilmektedir. Bunun sebebi ise hareketli ortalamaların geçmiş fiyat hareketlerinden oluşmasındandır.

#### 4.4.1. Satış

In [13]:
# Belirlenen çerçeveler ölçüsünde satışların her mağazadaki ürün bazına ortalamalarını hesaplıyoruz.
for days in [3, 7, 28]:
    df['rolling_sold_mean_{}'.format(days)] = df.groupby(
    ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(
    lambda x: x.rolling(window=days).mean()).astype(np.float16)

df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,delta_price_all_rel,delta_price_weekn-1,delta_price_cat_rel,sold_lag_1,sold_lag_8,sold_lag_21,sold_lag_34,item_sold_avg,state_sold_avg,store_sold_avg,cat_sold_avg,dept_sold_avg,cat_dept_sold_avg,store_item_sold_avg,cat_item_sold_avg,dept_item_sold_avg,state_store_sold_avg,state_store_cat_sold_avg,store_cat_dept_sold_avg,rolling_sold_mean_3,rolling_sold_mean_7,rolling_sold_mean_28
0,14370,1437,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.21960,1.23340,1.32324,0.56885,0.70557,0.70557,0.32617,0.21960,0.21960,1.32324,0.81348,1.03516,NaN,NaN,NaN
1,14380,1438,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26343,1.23340,1.32324,0.56885,0.70557,0.70557,0.25757,0.26343,0.26343,1.32324,0.81348,1.03516,NaN,NaN,NaN
2,14390,1439,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.07782,1.23340,1.32324,0.56885,0.70557,0.70557,0.15918,0.07782,0.07782,1.32324,0.81348,1.03516,NaN,NaN,NaN
3,14400,1440,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.04102,1.23340,1.32324,0.56885,0.70557,0.70557,1.71875,2.04102,2.04102,1.32324,0.81348,1.03516,NaN,NaN,NaN
4,14410,1441,3,1,0,0,1,0,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.76660,1.23340,1.32324,0.56885,0.70557,0.70557,0.97266,0.76660,0.76660,1.32324,0.81348,1.03516,NaN,NaN,NaN


#### 4.4.2. Lag

In [14]:
# Belirlenen çerçeveler ölçüsünde satışların lag'lı versiyonunu her mağazadaki ürün bazına ortalamalarını hesaplıyoruz.
for window, lag in zip([7, 7, 28, 28], [8, 34, 8, 34]):
    df['rolling_lag_{}_win_{}'.format(window, lag)] = df.groupby(
    ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold_lag_{}'.format(lag)].transform(
    lambda x: x.rolling(window=window).mean()).astype(np.float16)

### 4.5. Trends

#### 4.5.1. Mağaza Bağımlı Ürün

In [15]:
# Ürün satışlarının genel ortalaması hesaplanır.
df['avg_sold'] = df.groupby(
    ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform('mean').astype(np.float16)

# Günlük satıştan ortalama satışı çıkararak ürünün satış trendini hesaplıyoruz.
df['selling_trend'] = (df['sold'] - df['avg_sold']).astype(np.float16)

# Ortalama kolonunu silebiliriz.
df.drop(['avg_sold'],axis=1,inplace=True)

#### 4.5.2. Mağaza Bağımsız Ürün

In [16]:
# Mağaza bağımsız ürünün günlük satış ortalamasını hesaplıyoruz.
df['item_daily_avg_sold'] = df.groupby(
    ['id', 'item_id', 'dept_id', 'cat_id', 'd'])['sold'].transform('mean').astype(np.float16)
# Mağaza bağımsız ürünün toplam satış ortalamasını hesaplıyoruz.
df['item_avg_sold'] = df.groupby(
    ['id', 'item_id', 'dept_id', 'cat_id'])['sold'].transform('mean').astype(np.float16)
# Mağaza bağımsız ürünün satış trendini hesaplıyoruz.
df['item_selling_trend'] = (df['item_daily_avg_sold'] - df['item_avg_sold']).astype(np.float16)

# Gereksiz kolonları siliyoruz.
df.drop(['item_daily_avg_sold','item_avg_sold'],axis=1,inplace=True)

### 4.6. Rolling Max

In [17]:
# Belirlenen çerçeveler ölçüsünde satışların her mağazadaki ürün bazına maksimum değerini hesaplıyoruz.
for days in [ 7, 14, 28]:
    df['rolling_sold_max_{}'.format(days)] = df.groupby(
    ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(
    lambda x: x.rolling(window=days).max()).astype(np.float16)

### 4.7. Stock Out

In [18]:
# Sort the dataset by item_id and day
df.sort_values(['id', 'd'], ascending =[True, True], inplace = True)

# Mapping id change
df['id_change'] = df['id'].diff().fillna(0)
print("{:,} unique id with {:,} id changes".format(df.id.nunique(), df['id_change'].sum()))
# Zero Sale 
df['zero_sale'] = (df['sold'] == 0) * (df['id_change']==False) * 1
df['stock_out_id'] = (df['zero_sale'].transform(
        lambda x: x.rolling(window=3).sum()).astype(np.float16)==3) * 1

# Drop useless columns
df.drop(['id_change', 'zero_sale'], axis = 1, inplace = True)

# Stock-Outs in the last n days ?
for n_days in [1, 2, 7]: 
    df['stock_out_id_last_{}_days'.format(n_days)] = (df['stock_out_id'].transform(
        lambda x: x.rolling(window=n_days).sum()).astype(np.float16)>0) * 1

30,490 unique id with 30,489.0 id changes


### 4.8. Store Opening

In [19]:
# Store Closed = Sales zero
df['store_closed'] = (df.groupby(
    ['store_id', 'd'])['sold'].transform('sum').astype(np.float16) == 0) * 1

for n_days in [1, 2, 7]: # closed the last week 
    df['store_closed_last_{}_days'.format(n_days)] = (df['store_closed'].transform(
        lambda x: x.rolling(window=n_days).sum()).astype(np.float16)>0) * 1